In [1]:
import pandas as pd
import numpy as np
import os
from pathlib import Path

In [3]:
# Map of raw files we will use 
files = {
    "orders": RAW / "olist_orders_dataset.csv",
    "order_items": RAW / "olist_order_items_dataset.csv",
    "customers": RAW / "olist_customers_dataset.csv",
    "products": RAW / "olist_products_dataset.csv",
    "category_translation": RAW / "product_category_name_translation.csv",
    "payments": RAW / "olist_order_payments_dataset.csv",
    "reviews": RAW / "olist_order_reviews_dataset.csv",
    "sellers": RAW / "olist_sellers_dataset.csv",
    "geolocation": RAW / "olist_geolocation_dataset.csv",
}

In [4]:
# Helper: safe csv reader (with low_mem False and utf-8-sig fallback)
def read_csv_safe(path: Path) -> pd.DataFrame:
    try:
        return pd.read_csv(path, low_memory=False)
    except UnicodeDecodeError:
        return pd.read_csv(path, encoding="latin-1", low_memory=False)

In [5]:
loaded = {}
missing = []
for k, p in files.items():
    if p.exists():
        loaded[k] = read_csv_safe(p)
    else:
        missing.append((k, str(p)))
print(f"Loaded {len(loaded)} files, missing {len(missing)} files.")

Loaded 9 files, missing 0 files.


In [6]:
# Basic profiling function
def profile_df(name: str, df: pd.DataFrame) -> dict:
    info = {}
    info["dataset"] = name
    info["rows"] = len(df)
    info["cols"] = df.shape[1]
    info["memory_mb"] = round(df.memory_usage(deep=True).sum() / (1024**2), 2)
    # Null ratio
    nulls = df.isna().mean().round(3)
    info["null_cols_cnt"] = int((nulls > 0).sum())
    info["top_null_cols"] = ", ".join(nulls.sort_values(ascending=False).head(5).index.tolist())
    # Heuristic key candidates
    key_cols = [c for c in df.columns if c.endswith("_id")]
    info["id_cols"] = ", ".join(key_cols[:5])
    # Date-like cols
    date_cols = [c for c in df.columns if "date" in c or "timestamp" in c or "purchase" in c]
    info["date_cols"] = ", ".join(date_cols[:5])
    # Duplicates check for singular keys if obvious
    if name == "orders" and "order_id" in df.columns:
        info["dup_key_rows"] = int(df.duplicated("order_id").sum())
    elif name == "customers" and "customer_id" in df.columns:
        info["dup_key_rows"] = int(df.duplicated("customer_id").sum())
    elif name == "products" and "product_id" in df.columns:
        info["dup_key_rows"] = int(df.duplicated("product_id").sum())
    elif name == "sellers" and "seller_id" in df.columns:
        info["dup_key_rows"] = int(df.duplicated("seller_id").sum())
    elif name == "reviews" and "review_id" in df.columns:
        info["dup_key_rows"] = int(df.duplicated("review_id").sum())
    else:
        info["dup_key_rows"] = np.nan
    return info

In [7]:
profiles = []
for name, df in loaded.items():
    profiles.append(profile_df(name, df))
profile_df_table = pd.DataFrame(profiles)

In [8]:
profile_df_table.head()

,dataset,rows,cols,memory_mb,null_cols_cnt,top_null_cols,id_cols,date_cols,dup_key_rows
0,orders,99441,8,58.97,3,"order_delivered_customer_date, order_delivered...","order_id, customer_id","order_purchase_timestamp, order_delivered_carr...",0.0
1,order_items,112650,7,39.43,0,"order_id, order_item_id, product_id, seller_id...","order_id, order_item_id, product_id, seller_id",shipping_limit_date,NaN
2,customers,99441,5,29.62,0,"customer_id, customer_unique_id, customer_zip_...","customer_id, customer_unique_id",,0.0
3,products,32951,9,6.79,4,"product_category_name, product_description_len...",product_id,,0.0
4,category_translation,71,2,0.01,0,"product_category_name, product_category_name_e...",,,NaN


In [9]:
# Save quick profile
profile_path = CLEAN / "data_profile_overview.csv"
profile_df_table.to_csv(profile_path, index=False)

In [10]:
# === Basic cleaning/transforms focused on Product Marketing ===
# 1) Standardize date columns for orders
orders = loaded.get("orders")
if orders is not None:
    # Parse datetimes robustly; keep original cols too if needed.
    for col in [c for c in orders.columns if "date" in c or "timestamp" in c]:
        orders[col] = pd.to_datetime(orders[col], errors="coerce")
    # Keep only core cols useful for marketing
    keep_cols = [c for c in orders.columns if any(x in c for x in ["order_id","customer_id","status","purchase","approved","delivered","estimated"])]
    orders_clean = orders[keep_cols].copy()
    orders_clean.rename(columns=lambda x: x.replace("order_", ""),
                        inplace=True)  # optional: shorten names: id,status,purchase_timestamp,...
    orders_clean.to_csv(CLEAN / "orders_clean.csv", index=False)
else:
    orders_clean = None

In [11]:
orders_clean.head()

,id,customer_id,status,purchase_timestamp,approved_at,delivered_carrier_date,delivered_customer_date,estimated_delivery_date
0,e481f51cbdc54678b7cc49136f2d6af7,9ef432eb6251297304e76186b10a928d,delivered,2017-10-02 10:56:33,2017-10-02 11:07:15,2017-10-04 19:55:00,2017-10-10 21:25:13,2017-10-18
1,53cdb2fc8bc7dce0b6741e2150273451,b0830fb4747a6c6d20dea0b8c802d7ef,delivered,2018-07-24 20:41:37,2018-07-26 03:24:27,2018-07-26 14:31:00,2018-08-07 15:27:45,2018-08-13
2,47770eb9100c2d0c44946d9cf07ec65d,41ce2a54c0b03bf3443c3d931a367089,delivered,2018-08-08 08:38:49,2018-08-08 08:55:23,2018-08-08 13:50:00,2018-08-17 18:06:29,2018-09-04
3,949d5b44dbf5de918fe9c16f97b45f8a,f88197465ea7920adcdbec7375364d82,delivered,2017-11-18 19:28:06,2017-11-18 19:45:59,2017-11-22 13:39:59,2017-12-02 00:28:42,2017-12-15
4,ad21c59c0840e6cb83a9ceb5573f8159,8ab97904e6daea8866dbdbc4fb7aad2c,delivered,2018-02-13 21:18:39,2018-02-13 22:20:29,2018-02-14 19:46:34,2018-02-16 18:17:02,2018-02-26


In [12]:
orders_clean.shape

(99441, 8)

In [13]:
# 2) Standardize order_items
order_items = loaded.get("order_items")
if order_items is not None:
    # Ensure numeric types for price/freight
    for col in ["price","freight_value"]:
        if col in order_items.columns:
            order_items[col] = pd.to_numeric(order_items[col], errors="coerce")
    order_items_clean = order_items.copy()
    order_items_clean.to_csv(CLEAN / "order_items_clean.csv", index=False)
else:
    order_items_clean = None

In [14]:
order_items_clean.head()    

,order_id,order_item_id,product_id,seller_id,shipping_limit_date,price,freight_value
0,00010242fe8c5a6d1ba2dd792cb16214,1,4244733e06e7ecb4970a6e2683c13e61,48436dade18ac8b2bce089ec2a041202,2017-09-19 09:45:35,58.90,13.29
1,00018f77f2f0320c557190d7a144bdd3,1,e5f2d52b802189ee658865ca93d83a8f,dd7ddc04e1b6c2c614352b383efe2d36,2017-05-03 11:05:13,239.90,19.93
2,000229ec398224ef6ca0657da4fc703e,1,c777355d18b72b67abbeef9df44fd0fd,5b51032eddd242adc84c38acab88f23d,2018-01-18 14:48:30,199.00,17.87
3,00024acbcdf0a6daa1e931b038114c75,1,7634da152a4610f1595efa32f14722fc,9d7a1d34a5052409006425275ba1c2b4,2018-08-15 10:10:18,12.99,12.79
4,00042b26cf59d7ce69dfabb4e55b4fd9,1,ac6c3623068f30de03045865e4e10089,df560393f3a51e74553ab94004ba5c87,2017-02-13 13:57:51,199.90,18.14


In [15]:
order_items_clean.shape

(112650, 7)

In [16]:
# 3) Products + category translation
products = loaded.get("products")
cat_tr = loaded.get("category_translation")
if products is not None:
    products_clean = products.copy()
    # Merge translation if available
    if cat_tr is not None and "product_category_name" in products_clean.columns:
        products_clean = products_clean.merge(
            cat_tr.rename(columns={
                "product_category_name": "product_category_name",
                "product_category_name_english": "product_category_en"
            }),
            on="product_category_name", how="left"
        )
    products_clean.to_csv(CLEAN / "products_clean.csv", index=False)
else:
    products_clean = None

In [17]:
products_clean.head()   

,product_id,product_category_name,product_name_lenght,product_description_lenght,product_photos_qty,product_weight_g,product_length_cm,product_height_cm,product_width_cm,product_category_en
0,1e9e8ef04dbcff4541ed26657ea517e5,perfumaria,40.0,287.0,1.0,225.0,16.0,10.0,14.0,perfumery
1,3aa071139cb16b67ca9e5dea641aaa2f,artes,44.0,276.0,1.0,1000.0,30.0,18.0,20.0,art
2,96bd76ec8810374ed1b65e291975717f,esporte_lazer,46.0,250.0,1.0,154.0,18.0,9.0,15.0,sports_leisure
3,cef67bcfe19066a932b7673e239eb23d,bebes,27.0,261.0,1.0,371.0,26.0,4.0,26.0,baby
4,9dc1a7de274444849c219cff195d0b71,utilidades_domesticas,37.0,402.0,4.0,625.0,20.0,17.0,13.0,housewares


In [18]:
products_clean.shape

(32951, 10)

In [19]:
# 4) Customers + geolocation standardization
customers = loaded.get("customers")
geoloc = loaded.get("geolocation")
if customers is not None:
    customers_clean = customers.copy()
    # Normalize zip prefix column name across datasets
    # Try to detect the zip prefix column name
    cust_zip_col = None
    for c in customers_clean.columns:
        if "zip" in c and "prefix" in c:
            cust_zip_col = c
            break
    if cust_zip_col and geoloc is not None:
        geoloc_clean = geoloc.copy()
        # Detect geo zip column
        geo_zip_col = None
        for c in geoloc_clean.columns:
            if "zip" in c and "prefix" in c:
                geo_zip_col = c
                break
        if geo_zip_col:
            # Optionally keep only relevant geoloc cols
            keep_geo_cols = [geo_zip_col] + [c for c in geoloc_clean.columns if c.endswith("_city") or c.endswith("_state")]
            geoloc_min = geoloc_clean[keep_geo_cols].drop_duplicates()
            # Join for city/state enrichment
            customers_clean = customers_clean.merge(
                geoloc_min,
                left_on=cust_zip_col, right_on=geo_zip_col, how="left"
            )
    # Save
    customers_clean.to_csv(CLEAN / "customers_clean.csv", index=False)

In [20]:
customers_clean.shape

(148920, 8)

In [21]:
customers_clean.head()

,customer_id,customer_unique_id,customer_zip_code_prefix,customer_city,customer_state,geolocation_zip_code_prefix,geolocation_city,geolocation_state
0,06b8999e2fba1a1fbc88172c00ba8bc7,861eff4711a542e4b93843c6dd7febb0,14409,franca,SP,14409.0,franca,SP
1,18955e83d337fd6b2def6b18a428ac77,290c77bc529b7ac935b93aa66c333dc3,9790,sao bernardo do campo,SP,9790.0,sao bernardo do campo,SP
2,18955e83d337fd6b2def6b18a428ac77,290c77bc529b7ac935b93aa66c333dc3,9790,sao bernardo do campo,SP,9790.0,são bernardo do campo,SP
3,4e7b3e00288586ebd08712fdd0374a03,060e732b5b29e8181a18229c7b0b2b5e,1151,sao paulo,SP,1151.0,sao paulo,SP
4,4e7b3e00288586ebd08712fdd0374a03,060e732b5b29e8181a18229c7b0b2b5e,1151,sao paulo,SP,1151.0,são paulo,SP


In [22]:
# 5) Payments 
payments = loaded.get("payments")
if payments is not None:
    payments_clean = payments.copy()
    if "payment_value" in payments_clean.columns:
        payments_clean["payment_value"] = pd.to_numeric(payments_clean["payment_value"], errors="coerce")
    payments_clean_path = CLEAN / "payments_clean.csv"
    payments_clean.to_csv(payments_clean_path, index=False)
   

In [23]:
payments_clean.shape    

(103886, 5)

In [24]:
payments_clean.head()

,order_id,payment_sequential,payment_type,payment_installments,payment_value
0,b81ef226f3fe1789b1e8b2acac839d17,1,credit_card,8,99.33
1,a9810da82917af2d9aefd1278f1dcfa0,1,credit_card,1,24.39
2,25e8ea4e93396b6fa0d3dd708e76c1bd,1,credit_card,1,65.71
3,ba78997921bbcdc1373bb41e913ab953,1,credit_card,8,107.78
4,42fdf880ba16b47b59251dd489d4441a,1,credit_card,2,128.45


In [25]:
# 6) Reviews 
reviews = loaded.get("reviews")
if reviews is not None:
    reviews_clean = reviews.copy()
    for col in [c for c in reviews_clean.columns if any(x in c for x in ["date", "timestamp", "creation"])]:
        reviews_clean[col] = pd.to_datetime(reviews_clean[col], errors="coerce")
    reviews_clean_path = CLEAN / "reviews_clean.csv"
    reviews_clean.to_csv(reviews_clean_path, index=False)

In [26]:
reviews_clean.shape 

(99224, 7)

In [27]:
reviews_clean.head()

,review_id,order_id,review_score,review_comment_title,review_comment_message,review_creation_date,review_answer_timestamp
0,7bc2406110b926393aa56f80a40eba40,73fc7af87114b39712e6da79b0a377eb,4,NaN,NaN,2018-01-18,2018-01-18 21:46:59
1,80e641a11e56f04c1ad469d5645fdfde,a548910a1c6147796b98fdf73dbeba33,5,NaN,NaN,2018-03-10,2018-03-11 03:05:13
2,228ce5500dc1d8e020d8d1322874b6f0,f9e4b658b201a9f2ecdecbb34bed034b,5,NaN,NaN,2018-02-17,2018-02-18 14:36:24
3,e64fb393e7b32834bb789ff8bb30750e,658677c97b385a9be170737859d3511b,5,NaN,Recebi bem antes do prazo estipulado.,2017-04-21,2017-04-21 22:02:06
4,f7c4243c7fe1938f181bec41a392bdeb,8e6bfb81e283fa7e4f11123a3fb894f1,5,NaN,Parabéns lojas lannister adorei comprar pela I...,2018-03-01,2018-03-02 10:26:53


In [32]:
# === Join coverage checks ===
checks = []

# order_items -> orders
if (order_items_clean is not None) and (orders_clean is not None):
    coverage = order_items_clean["order_id"].isin(orders_clean["id"]).mean().round(3)
    checks.append({"join": "order_items -> orders", "coverage_ratio": coverage})

# order_items -> products
if (order_items_clean is not None) and (products_clean is not None):
    coverage = order_items_clean["product_id"].isin(products_clean["product_id"]).mean().round(3)
    checks.append({"join": "order_items -> products", "coverage_ratio": coverage})

# payments -> orders
if (payments is not None) and (orders_clean is not None):
    coverage = payments["order_id"].isin(orders_clean["id"]).mean().round(3)
    checks.append({"join": "payments -> orders", "coverage_ratio": coverage})

# reviews -> orders
if (reviews is not None) and (orders_clean is not None):
    coverage = reviews["order_id"].isin(orders_clean["id"]).mean().round(3)
    checks.append({"join": "reviews -> orders", "coverage_ratio": coverage})

join_checks = pd.DataFrame(checks)
join_checks.head()

,join,coverage_ratio
0,order_items -> orders,1.0
1,order_items -> products,1.0
2,payments -> orders,1.0
3,reviews -> orders,1.0


Key consistency checks between tables:
   - order_items → orders : verify if all items are linked to an existing order
   - order_items → products : verify if all items are linked to an existing product
   - payments → orders : verify if all payments are linked to an existing order
   - reviews → orders : verify if all reviews are linked to an existing order

In [ ]:
# PROTOTYPE "gold-like" MART (order line level)
if all([
    order_items_clean is not None,
    orders_clean is not None,
    products_clean is not None
]):
    oi = order_items_clean[["order_id", "order_item_id", "product_id", "seller_id", "price", "freight_value"]].copy()

    o = orders_clean[["id", "status", "purchase_timestamp", "delivered_customer_date", "estimated_delivery_date", "customer_id"]].copy()
    o = o.rename(columns={"id": "order_id"})

    p_cols = ["product_id", "product_category_name"]
    if "product_category_en" in products_clean.columns:
        p_cols += ["product_category_en"]
    p = products_clean[p_cols].copy()

    gold = oi.merge(o, on="order_id", how="left").merge(p, on="product_id", how="left")
    # basic derived metric
    gold["revenue"] = gold["price"].fillna(0) + gold["freight_value"].fillna(0)

gold.shape
gold.head()

,order_id,order_item_id,product_id,seller_id,price,freight_value,status,purchase_timestamp,delivered_customer_date,estimated_delivery_date,customer_id,product_category_name,product_category_en,revenue
0,00010242fe8c5a6d1ba2dd792cb16214,1,4244733e06e7ecb4970a6e2683c13e61,48436dade18ac8b2bce089ec2a041202,58.90,13.29,delivered,2017-09-13 08:59:02,2017-09-20 23:43:48,2017-09-29,3ce436f183e68e07877b285a838db11a,cool_stuff,cool_stuff,72.19
1,00018f77f2f0320c557190d7a144bdd3,1,e5f2d52b802189ee658865ca93d83a8f,dd7ddc04e1b6c2c614352b383efe2d36,239.90,19.93,delivered,2017-04-26 10:53:06,2017-05-12 16:04:24,2017-05-15,f6dd3ec061db4e3987629fe6b26e5cce,pet_shop,pet_shop,259.83
2,000229ec398224ef6ca0657da4fc703e,1,c777355d18b72b67abbeef9df44fd0fd,5b51032eddd242adc84c38acab88f23d,199.00,17.87,delivered,2018-01-14 14:33:31,2018-01-22 13:19:16,2018-02-05,6489ae5e4333f3693df5ad4372dab6d3,moveis_decoracao,furniture_decor,216.87
3,00024acbcdf0a6daa1e931b038114c75,1,7634da152a4610f1595efa32f14722fc,9d7a1d34a5052409006425275ba1c2b4,12.99,12.79,delivered,2018-08-08 10:00:35,2018-08-14 13:32:39,2018-08-20,d4eb9395c8c0431ee92fce09860c5a06,perfumaria,perfumery,25.78
4,00042b26cf59d7ce69dfabb4e55b4fd9,1,ac6c3623068f30de03045865e4e10089,df560393f3a51e74553ab94004ba5c87,199.90,18.14,delivered,2017-02-04 13:57:51,2017-03-01 16:42:31,2017-03-17,58dbd0b2d70206bf40e62cd34e84d795,ferramentas_jardim,garden_tools,218.04


In [35]:
gold.describe()

,order_item_id,price,freight_value,purchase_timestamp,delivered_customer_date,estimated_delivery_date,revenue
count,112650.000000,112650.000000,112650.000000,112650,110196,112650,112650.000000
mean,1.197834,120.653739,19.990320,2018-01-01 00:09:48.464376576,2018-01-14 13:25:24.023939328,2018-01-24 20:12:15.531291392,140.644059
min,1.000000,0.850000,0.000000,2016-09-04 21:15:19,2016-10-11 13:46:32,2016-10-04 00:00:00,6.080000
25%,1.000000,39.900000,13.080000,2017-09-13 19:17:04,2017-09-26 20:09:44.500000,2017-10-05 00:00:00,55.220000
50%,1.000000,74.990000,16.260000,2018-01-19 23:02:16,2018-02-02 20:57:23,2018-02-16 00:00:00,92.320000
75%,1.000000,134.900000,21.150000,2018-05-04 17:30:36.750000128,2018-05-15 20:09:21.500000,2018-05-28 00:00:00,157.937500
max,21.000000,6735.000000,409.680000,2018-09-03 09:06:57,2018-10-17 13:22:46,2018-10-25 00:00:00,6929.310000
std,0.705124,183.633928,15.806405,NaN,NaN,NaN,190.724394


In [36]:
gold.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 112650 entries, 0 to 112649
Data columns (total 14 columns):
 #   Column                   Non-Null Count   Dtype         
---  ------                   --------------   -----         
 0   order_id                 112650 non-null  object        
 1   order_item_id            112650 non-null  int64         
 2   product_id               112650 non-null  object        
 3   seller_id                112650 non-null  object        
 4   price                    112650 non-null  float64       
 5   freight_value            112650 non-null  float64       
 6   status                   112650 non-null  object        
 7   purchase_timestamp       112650 non-null  datetime64[ns]
 8   delivered_customer_date  110196 non-null  datetime64[ns]
 9   estimated_delivery_date  112650 non-null  datetime64[ns]
 10  customer_id              112650 non-null  object        
 11  product_category_name    111047 non-null  object        
 12  product_category

In [41]:

unique_counts = gold.nunique()
print("Count of unique values in each column:")
print(unique_counts)

print("\n" + "="*50)
print("In details: ")
print("="*50)

unique_analysis = pd.DataFrame({
    'column': gold.columns,
    'total_rows': len(gold),
    'unique_values': gold.nunique(),
    'uniqueness_ratio': (gold.nunique() / len(gold) * 100).round(2)
})

print(unique_analysis)

Count of unique values in each column:
order_id                   98666
order_item_id                 21
product_id                 32951
seller_id                   3095
price                       5968
freight_value               6999
status                         7
purchase_timestamp         98112
delivered_customer_date    95664
estimated_delivery_date      450
customer_id                98666
product_category_name         73
product_category_en           71
revenue                    29149
dtype: int64

In details: 
                                          column  total_rows  unique_values  \
order_id                                order_id      112650          98666   
order_item_id                      order_item_id      112650             21   
product_id                            product_id      112650          32951   
seller_id                              seller_id      112650           3095   
price                                      price      112650           5968  

In [42]:

gold_path = CLEAN / "sales_mart_marketing_prototype.csv"
gold.to_csv(gold_path, index=False)
print(f"[SAVE] {gold_path}")

[SAVE] c:\Users\daria\Documents\Data_market1\data\clean\sales_mart_marketing_prototype.csv


- Pour aller plus loin:
   * Robust data quality checks (unique keys, referential integrity).
   * Normalize geo to consistent region levels (city/state).
   * Add campaign data if available; compute post-campaign lift KPIs.
   * Basket analysis (association rules) on the gold table.
   * Schedule pipeline (CI, cron, or lightweight orchestrator).